In [1]:
!git clone https://github.com/TapasKumarDutta1/circle_detection.git

Cloning into 'circle_detection'...
remote: Enumerating objects: 30, done.
remote: Counting objects: 100% (30/30), done.
remote: Compressing objects: 100% (25/25), done.
remote: Total 30 (delta 6), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (30/30), 9.85 KiB | 4.92 MiB/s, done.
Resolving deltas: 100% (6/6), done.


In [2]:
import sys
sys.path.append('/content/circle_detection')

In [3]:
from generate_dataset import generate
from torchvision import models
from torch import nn
import glob
import numpy as np
from torch.utils.data import DataLoader
from torchvision import transforms
from dataset import CustomDataset
from torch import optim
from torch.optim import lr_scheduler
import torch
from evaluation import check_iou
generate(10000)


80it [00:00, 773.95it/s]

Using parameters: noise_level=0.5, img_size=100, min_radius=10, max_radius=50, dataset_path='ds'


12999it [00:16, 765.57it/s] 


In [4]:
import os
path_of_the_directory = "/content/train/labels"
ext = ".txt"
files_train = []
loc_x_train = []
loc_y_train = []
radiuses_train = []
for file in os.listdir(path_of_the_directory):
    if file.endswith(ext):
        f = open(os.path.join(path_of_the_directory, file), "r")
        files_train.append(
            os.path.join(path_of_the_directory, file)
            .replace("labels", "images")
            .replace(".txt", "")
        )
        loc_x_train.append(int(f.readline()))
        loc_y_train.append(int(f.readline().replace("\n", "")))
        radiuses_train.append(int(f.readline().replace("\n", "")))


In [5]:
path_of_the_directory = "/content/valid/labels"
ext = ".txt"
files_valid = []
loc_x_valid = []
loc_y_valid = []
radiuses_valid = []
for file in os.listdir(path_of_the_directory):
    if file.endswith(ext):
        f = open(os.path.join(path_of_the_directory, file), "r")
        files_valid.append(
            os.path.join(path_of_the_directory, file)
            .replace("labels", "images")
            .replace(".txt", "")
        )
        loc_x_valid.append(int(f.readline()))
        loc_y_valid.append(int(f.readline().replace("\n", "")))
        radiuses_valid.append(int(f.readline().replace("\n", "")))


In [6]:
path_of_the_directory = "/content/test/labels"
ext = ".txt"
files_test = []
loc_x_test = []
loc_y_test = []
radiuses_test = []
for file in os.listdir(path_of_the_directory):
    if file.endswith(ext):
        f = open(os.path.join(path_of_the_directory, file), "r")
        files_test.append(
            os.path.join(path_of_the_directory, file)
            .replace("labels", "images")
            .replace(".txt", "")
        )
        loc_x_test.append(int(f.readline()))
        loc_y_test.append(int(f.readline().replace("\n", "")))
        radiuses_test.append(int(f.readline().replace("\n", "")))


In [7]:
ls = []
for i in glob.glob("/content/train/images/*.npy"):
    ls.append(np.load(i, allow_pickle=True))
mean, std = np.mean(np.stack(ls)), np.std(np.stack(ls))

transform = transforms.Compose(
    [transforms.ToTensor(), transforms.Normalize((mean, mean, mean), (std, std, std))]
)

test_transform = transforms.Compose(
    [transforms.ToTensor(), transforms.Normalize((mean, mean, mean), (std, std, std))]
)

batch_size = 10

dataset_stages = ["train", "val", "test"]
image_datasets = {
    "train": CustomDataset(
        files_train, loc_x_train, loc_y_train, radiuses_train, batch_size, transform
    ),
    "val": CustomDataset(
        files_valid,
        loc_x_valid,
        loc_y_valid,
        radiuses_valid,
        batch_size,
        test_transform,
    ),
    "test": CustomDataset(
        files_test, loc_x_test, loc_y_test, radiuses_test, batch_size, test_transform
    ),
}

dataloaders = {
    x: DataLoader(
        image_datasets[x],
        batch_size=image_datasets[x].BatchSize,
        shuffle=False,
        num_workers=0,
    )
    for x in dataset_stages
}
dataset_sizes = {x: len(image_datasets[x]) for x in ["train", "val"]}
model = models.shufflenet_v2_x1_0()
model.fc = nn.Linear(in_features=1024, out_features=3, bias=True)
criterion = nn.MSELoss()

optimizer = optim.Adam(model.parameters(), lr=0.01)

scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, "min")
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)


In [8]:
from training import train_model
model, _ = train_model(
    model,
    dataloaders["train"],
    dataloaders["val"],
    criterion,
    optimizer,
    scheduler,
    device,
    50,
)


100%|██████████| 100/100 [00:02<00:00, 41.77it/s]


Epoch [0], train_loss: 0.0541, val_loss: 0.0040


100%|██████████| 100/100 [00:01<00:00, 55.01it/s]


Epoch [1], train_loss: 0.0053, val_loss: 0.0033


100%|██████████| 100/100 [00:01<00:00, 54.51it/s]


...

Epoch [19], train_loss: 0.0007, val_loss: 0.0017


100%|██████████| 100/100 [00:01<00:00, 51.72it/s]


Epoch [20], train_loss: 0.0016, val_loss: 0.0013


100%|██████████| 100/100 [00:01<00:00, 55.58it/s]


Epoch [21], train_loss: 0.0006, val_loss: 0.0011
Early stopping after 22 epochs without improvement.
Training complete in 15m 36s


In [9]:
check_iou(model, dataloaders['test'], device)

0.9895

In [10]:
ls=[]
for i in range(50,95,5):
  ls.append(check_iou(model, dataloaders['test'], device,i/100))
np.mean(ls)

0.8203333333333334